## Вывод полуаналитических зависимостей для метрики $d_{\text{max}}$ (изохронное отклонение)

Вид линий уровня для графика $r_{min}(\sigma_{pos}, \sigma_{vel})$ поддатлкивает на мысль о том, что зависимость должна быть такой:
$$
r_{min} = \sqrt[n]{\alpha_1\cdot\sigma_{pos}^n+\alpha_2\cdot\sigma_{vel}^n}
$$

Для нахождения коэффициентов $\alpha = (\alpha_1, \alpha_2)$ воспользуемся формулой функционала для линейной регресии:
$$
J_2 = \dfrac{1}{N}\sum_{i=1}^n(r_{min}^n-\alpha_1\cdot\sigma_{pos}^n-\alpha_2\cdot\sigma_{vel}^n)^2 \longrightarrow \min_{\alpha_1, \alpha_2}
$$

Для такого функционала есть ответ. Если переписать его ядро в матричном виде, то по формуле Маркова-Гаусса:
$$
\boxed{\alpha^*=(A^TA)^{-1}A^Ty} = \alpha^*(n)
$$

In [1]:
import numpy as np

In [2]:
from utils.libration_sense import get_xf, km2du, kmS2vu, du2km, get_maxdeviation_wo_integrate
from utils.formula_creators import n_finder, alpha_finder_of_n

In [3]:
orbit_type = "L1"
orbit_number = 192

In [ ]:
from IPython.display import display, Math

# ищем все параметры для формулы
xfinal = get_xf(orbit_type, orbit_number)
n, A_normed, y_du = n_finder(orbit_type, orbit_number, xfinal, grid_density=40, seed=42, amount_of_points=10_000)
alpha, deviation_max = alpha_finder_of_n(A_normed, y_du, n)

print('Степень n = ', n, sep='')
print("alpha1, alpha2 = ", alpha)

progress: 100%|██████████| 1600/1600 [16:58<00:00,  1.57it/s]

Степень n = 2.217977546021809
alpha1, alpha2 =  [0.16411655 0.83937246]


In [4]:
def deviation_analytic(n, alpha, std_pos_normed, std_vel_normed):
    return np.power(alpha[0]*np.power(std_pos_normed, n) + alpha[1]*np.power(std_vel_normed, n), 1/n)


formula = f"d = \\sqrt[{np.around(n, 4)}]{{{np.around(alpha[0], 2)}\\cdot \\sigma_{{pos}}^{{{np.around(n, 4)}}} + {np.around(alpha[1], 2)}\\cdot \\sigma_{{vel}}^{{{np.around(n, 4)}}}}}\\cdot{np.around(du2km(deviation_max), 3)} \\ [km]"

std_pos, std_vel = km2du(0.4), kmS2vu(0.005e-3)
std_pos_max, std_vel_max = km2du(1), kmS2vu(0.01e-3)

print(f"Пробная точка: [sigma_pos = {4} км, sigma_vel = {0.03} м/с]")
print(f"Крайние значения данных: [sigma_pos_max = {8} км, sigma_vel_max = {0.05} м/с]")
print('Точное значение: ', du2km(get_maxdeviation_wo_integrate('L2', 290, xfinal, std_pos, std_vel, amount_of_points=15000)))
print('Значение, вычисленное по аппроксимирующей формуле: ', deviation_analytic(2, alpha, std_pos/std_pos_max, std_vel/std_vel_max) * du2km(deviation_max))

Пробная точка: [sigma_pos = 4 км, sigma_vel = 0.03 м/с]
Крайние значения данных: [sigma_pos_max = 8 км, sigma_vel_max = 0.05 м/с]
Точное значение:  51.08379666989129
Значение, вычисленное по аппроксимирующей формуле:  49.25081572483868
